In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# set log level to debug
import logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

In [7]:
from task import VolcengineMLTaskManager

In [8]:
volcengine_task_manager = VolcengineMLTaskManager('config/volc_mltask.yaml')

2024-02-28 11:26:37,293 - task.base - DEBUG - VolcengineMLTaskManager is shutdown


In [9]:
t = volcengine_task_manager.submit(
    name='script_test',
    entrypoint_path='./hello.sh')
volcengine_task_manager.wait(t)

2024-02-28 11:26:41,776 - tos.auth - DEBUG - pre-request: canonical_request:
PUT
/scripts/66adc1cc3389771f1786e32ad33cfb65588cb4e24a7cc0bbf2ad15289770b295

content-type:binary/octet-stream
host:xtalpi-mlp.tos-cn-beijing.volces.com
x-tos-content-sha256:UNSIGNED-PAYLOAD
x-tos-date:20240228T032641Z

content-type;host;x-tos-content-sha256;x-tos-date
UNSIGNED-PAYLOAD
2024-02-28 11:26:41,780 - tos.auth - DEBUG - pre-request: string_to_sign:
TOS4-HMAC-SHA256
20240228T032641Z
20240228/cn-beijing/tos/request
bbb0242aee52ec0f067cb5c4b0a0c46814543dbe0b595712c63bf79ba8c5d119
2024-02-28 11:26:41,784 - tos.auth - DEBUG - pre-request: signature:
93d89c75ad63da7fd20cbd481cb4efd84cdf48cb915827abf3bcd7d09f71e360
2024-02-28 11:26:41,788 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): xtalpi-mlp.tos-cn-beijing.volces.com:443
2024-02-28 11:26:42,063 - urllib3.connectionpool - DEBUG - https://xtalpi-mlp.tos-cn-beijing.volces.com:443 "PUT /scripts/66adc1cc3389771f1786e32ad33cfb65588cb4e

<TaskStatus.COMPLETED: 3>

In [7]:
volcengine_task_manager.list()

2024-02-28 10:13:35,554 - urllib3.connectionpool - DEBUG - http://open.volcengineapi.com:80 "POST /?Action=ListCustomTasks&Version=2021-10-01 HTTP/1.1" 200 None


,name,entrypoint_path,status
task_id,,,
20240228-101220-0280,script_test,cat /root/code/hello.sh,TaskStatus.COMPLETED


In [15]:
volcengine_task_manager._task_client.get_custom_task('t-20240227172005-cvbx6')['Result']

2024-02-28 10:06:19,819 - urllib3.connectionpool - DEBUG - Resetting dropped connection: open.volcengineapi.com
2024-02-28 10:06:20,063 - urllib3.connectionpool - DEBUG - http://open.volcengineapi.com:80 "POST /?Action=GetCustomTask&Version=2021-10-01 HTTP/1.1" 200 None


{'Id': 't-20240227172005-cvbx6',
 'Name': 'hello_sh',
 'CreateTime': '2024-02-27T09:20:05Z',
 'LaunchTime': '2024-02-27T09:20:28Z',
 'FinishTime': '2024-02-27T09:20:48Z',
 'CreatorUserId': 22571628,
 'EnableTensorBoard': False,
 'State': 'Failed',
 'CodeSource': '',
 'CodeOriPath': '',
 'ImageSpec': {'Type': 'Preset',
  'Url': 'vemlp-cn-beijing.cr.volces.com/preset-images/slurm:0.7.0'},
 'TOSCodePath': 'tos://ml-platform-auto-created-required-2100205915-cn-beijing/static/customtask/manifest/snapshot_ljgs9jm4hpsskt67.manifest',
 'LocalCodePath': '/root/code',
 'EntrypointPath': '/root/code/hello.sh',
 'Args': '',
 'Envs': [{'Name': 'MLP_SLURM_WORKER_CPU', 'Value': '22'},
  {'Name': 'MLP_SLURM_WORKER_MEM', 'Value': '92160'},
  {'Name': 'MLP_SLURM_WORKER_GPU', 'Value': '1'},
  {'Name': 'MLP_SLURM_WORKER_GPU_TYPE', 'Value': 'GPU'},
  {'Name': 'MLP_SLURM_PARTITION', 'Value': 'q-20240226170714-vsgh9'},
  {'Name': 'MLP_SLURM_ARGS_FROM_CLI', 'Value': ''},
  {'Name': 'MLP_SLURM_CLUSTER_MODE', '

In [13]:
volcengine_task_manager.submit(
    name='test-script',
    entrypoint_path='./hello.sh'
)

'20240227-201125-2176'

2024-02-27 20:11:25,512 - tos.auth - DEBUG - pre-request: canonical_request:
PUT
/scripts/entrypoint.sh

content-type:text/x-sh
host:xtalpi-mlp.tos-cn-beijing.volces.com
x-tos-content-sha256:UNSIGNED-PAYLOAD
x-tos-date:20240227T121125Z

content-type;host;x-tos-content-sha256;x-tos-date
UNSIGNED-PAYLOAD
2024-02-27 20:11:25,513 - tos.auth - DEBUG - pre-request: string_to_sign:
TOS4-HMAC-SHA256
20240227T121125Z
20240227/cn-beijing/tos/request
af4af8787fa475a14d4d20323eb087a2950cff48f73572cb00dc5be103786e6f
2024-02-27 20:11:25,514 - tos.auth - DEBUG - pre-request: signature:
8e3dd8b06933966bb048819a0a7814f89be622e379c394864356aebe71fc945d
2024-02-27 20:11:25,518 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): xtalpi-mlp.tos-cn-beijing.volces.com:443
2024-02-27 20:11:25,722 - urllib3.connectionpool - DEBUG - https://xtalpi-mlp.tos-cn-beijing.volces.com:443 "PUT /scripts/entrypoint.sh HTTP/1.1" 200 0
2024-02-27 20:11:25,725 - tos.clientv2 - DEBUG - in-request: method:PU

In [7]:
volcengine_task_manager.list()

2024-02-27 20:03:48,179 - urllib3.connectionpool - DEBUG - http://open.volcengineapi.com:80 "POST /?Action=ListCustomTasks&Version=2021-10-01 HTTP/1.1" 200 None


,name,entrypoint_path,status
task_id,,,
20240227-200331-8603,test-script,./hello.sh,TaskStatus.RUNNING


In [5]:
volcengine_task_manager.submit(
    name='colabfold_db',
    entrypoint_path='MMSEQS_NO_INDEX=1 /root/code/setup_databases.sh /vepfs-mlp/hongning.zhang/colabfold_db',
    tos_code_path='tos://xtalpi-mlp/hongning.zhang/code/download_colabfold_db.sh',
    local_code_path='/root/code/setup_database.sh'
)

'20240227-182319-1524'

In [8]:
credentials = volcengine_task_manager._task_client.service_info.credentials

In [9]:
tos_client = tos.TosClientV2(
    ak=credentials.ak,
    sk=credentials.sk,
    endpoint=f'tos-{credentials.region}.volces.com',
    region=credentials.region
)

In [ ]:
tos_client

In [11]:
list_bucket = tos_client.list_buckets()

2024-02-27 19:02:27,004 - tos.utils - INFO - after-request: list_buckets exec httpCode: 200, requestId: d6f601ddc14361c165ddc143-b8dd259-1revEB-LB-cb-tos-front-azc-1, usedTime: 0.04086022451519966 s


In [16]:
bucket = list_bucket.buckets[3]

In [27]:
object_output = tos_client.get_object(
    bucket=bucket.name,
    key='hongning.zhang/code/download_colabfold_db.sh',
)

2024-02-27 19:13:36,243 - tos.utils - INFO - after-request: get_object exec httpCode: 200, requestId: e99801ddc3e0f77565ddc3e0-ac15c551-1revOy-GO-cb-tos-front-aza-1, usedTime: 0.18148407712578773 s


In [26]:
tos_client.get_object_to_file(
    bucket.name,
    'hongning.zhang/code/download_colabfold_db.sh',
    'download_colabfold_db.sh'
)

2024-02-27 19:10:23,248 - tos.utils - INFO - after-request: get_object exec httpCode: 200, requestId: bb6501ddc31f9e8065ddc31f-b45b8f2-1revLr-GO-cb-tos-front-azb-1, usedTime: 0.23932785540819168 s


In [28]:
object_output.read()

b'#!/bin/bash -ex\n# Setup everything for using mmseqs locally\n# Set MMSEQS_NO_INDEX to skip the index creation step (not useful for colabfold_search in most cases)\nARIA_NUM_CONN=8\nWORKDIR="${1:-$(pwd)}"\n\nPDB_SERVER="${2:-"rsync.wwpdb.org::ftp"}"\nPDB_PORT="${3:-"33444"}"\n# do initial download of the PDB through aws?\n# still syncs latest structures through rsync\nPDB_AWS_DOWNLOAD="${4:-}"\nPDB_AWS_SNAPSHOT="20240101"\n\nUNIREF30DB="uniref30_2302"\nMMSEQS_NO_INDEX=${MMSEQS_NO_INDEX:-}\ncd "${WORKDIR}"\n\nhasCommand () {\n    command -v "$1" >/dev/null 2>&1\n}\n\nSTRATEGY=""\nif hasCommand aria2c; then STRATEGY="$STRATEGY ARIA"; fi\nif hasCommand curl;   then STRATEGY="$STRATEGY CURL"; fi\nif hasCommand wget;   then STRATEGY="$STRATEGY WGET"; fi\nif [ "$STRATEGY" = "" ]; then\n\t    fail "No download tool found in PATH. Please install aria2c, curl or wget."\nfi\n\nif [ -n "${PDB_AWS_DOWNLOAD}" ]; then\n    if ! hasCommand aws; then\n        fail "aws command not found in PATH. Ple

In [28]:
entrypoints_paths = [
    'wget https://tos-tools.tos-cn-beijing.ivolces.com/linux/tosutil -O /vepfs-mlp/hongning.zhang/software/volc/bin/tosutil',
    'chmod +x /vepfs-mlp/hongning.zhang/software/volc/bin/tosutil',
    'export PATH=$PATH:/vepfs-mlp/hongning.zhang/software/volc/bin',
    f'tosutil config -i {credentials.ak} -k {credentials.sk} -re {credentials.region} -e tos-{credentials.region}.ivolces.com',
    f'tosutil cp tos://xtalpi-mlp/hongning.zhang/msa /vepfs-mlp/hongning.zhang/msa -r',
]
volcengine_task_manager.submit(
    name='load_msa_to_vepfs',
    entrypoint_path=' && '.join(entrypoints_paths),
    # image_id='hsbj-registry-cn-beijing.cr.volces.com/xtalpi-mlp/colabfold:v1'
)

'20240227-160209-0201'

In [31]:
entrypoints_paths = [
    'colabfold_batch --num-models 5 --num-recycle 10 --model-type alphafold2_multimer_v3  /vepfs-mlp/hongning.zhang/msa  /vepfs-mlp/hongning.zhang/output',
    f'tosutil config -i {credentials.ak} -k {credentials.sk} -re {credentials.region} -e tos-{credentials.region}.ivolces.com',
    'tosutil cp /vepfs-mlp/hongning.zhang/output tos://xtalpi-mlp/hongning.zhang/output -r'
]
volcengine_task_manager.submit(
    name='colabfold',
    entrypoint_path=' && '.join(entrypoints_paths),
    image_id='hsbj-registry-cn-beijing.cr.volces.com/xtalpi-mlp/colabfold:v1'
)

'20240227-163850-7904'

In [30]:
volcengine_task_manager.submit(
    name='colabfold_db',
    entrypoint_path='MMSEQS_NO_INDEX=1 /vepfs-mlp/hongning.zhang/setup_databases.sh /vepfs-mlp/hongning.zhang/colabfold_db',
)

'20240227-193446-3701'

In [34]:
entrypoints_paths = [
    'ln -s /vepfs-mlp/hongning.zhang/.cache /root/.cache',
    'python -m colabfold.download'
]
volcengine_task_manager.submit(
    name='colabfold_download',
    entrypoint_path=' && '.join(entrypoints_paths),
    image_id='hsbj-registry-cn-beijing.cr.volces.com/xtalpi-mlp/colabfold:v1'
)

'20240227-171122-7911'

In [23]:
task_id1 = volcengine_task_manager.submit(
    'test_task_1',
    'nvidia-smi; ls /vepfs-mlp/hongning.zhang -hla; exit 1'
)
task_id2 = volcengine_task_manager.submit(
    'test_task_2',
    'nvidia-smi; ls /vepfs-mlp/hongning.zhang -hla; echo ok',
    dependencies={
        'afternotok': task_id1
    }
)
volcengine_task_manager.wait(task_id1)

2024-02-26 19:28:58,328 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): open.volcengineapi.com:80
2024-02-26 19:29:00,422 - urllib3.connectionpool - DEBUG - http://open.volcengineapi.com:80 "POST /?Action=CreateCustomTask&Version=2021-10-01 HTTP/1.1" 200 None
2024-02-26 19:29:00,523 - urllib3.connectionpool - DEBUG - http://open.volcengineapi.com:80 "POST /?Action=GetCustomTask&Version=2021-10-01 HTTP/1.1" 200 None
2024-02-26 19:29:00,659 - urllib3.connectionpool - DEBUG - http://open.volcengineapi.com:80 "POST /?Action=GetCustomTask&Version=2021-10-01 HTTP/1.1" 200 None
2024-02-26 19:29:00,662 - task.base - DEBUG - Task 20240226-192858-8171 is TaskStatus.PENDING, waiting for afternotok to submit task 20240226-192858-5277
2024-02-26 19:29:01,577 - urllib3.connectionpool - DEBUG - http://open.volcengineapi.com:80 "POST /?Action=GetCustomTask&Version=2021-10-01 HTTP/1.1" 200 None
2024-02-26 19:29:01,722 - urllib3.connectionpool - DEBUG - http://open.volcengineapi.com

<TaskStatus.FAILED: 4>

In [8]:
from task.base import TaskStatus

task_id1 = volcengine_task_manager.submit(
    name="test_depes_afterok_1",
    entrypoint_path="nvidia-smi; sleep 5; echo ok",
)
task_id2 = volcengine_task_manager.submit(
    name="test_depes_afterok_2",
    entrypoint_path="nvidia-smi; sleep 1; echo ok",
    dependencies={"afterok": task_id1}
)
assert task_id1 is not None
assert task_id2 is not None
assert volcengine_task_manager.wait(
    task_id1, waiting_status=(TaskStatus.PENDING,)) == TaskStatus.RUNNING

In [9]:
assert volcengine_task_manager.status(task_id2) == TaskStatus.PENDING
assert volcengine_task_manager.wait(task_id2) == TaskStatus.COMPLETED

In [3]:
from task.slurm import SlurmTaskManager
slurm_task_manager = SlurmTaskManager('config/slurm_task.yaml')

In [60]:
jobid=slurm_task_manager.submit(
    'test_task',
    'nvidia-smi; sleep 80; exit 1',
    partitions='gpu',
)

In [65]:
jobid=slurm_task_manager.submit(
    'test_task',
    'nvidia-smi',
    partitions='gpu',
    dependencies={
        'after': 42670
    }
)

In [66]:
slurm_task_manager.list()

,name,entrypoint_path,status
task_id,,,
42670,test_task,nvidia-smi; sleep 80; exit 1,TaskStatus.FAILED
42671,test_task,nvidia-smi,TaskStatus.PENDING
42672,test_task,nvidia-smi,TaskStatus.COMPLETED


In [68]:
slurm_task_manager.cancel(42671)

In [ ]:
slurm_task_manager.cancel(jobid)
slurm_task_manager.list()
slurm_task_manager.wait(jobid)

In [6]:
task_id = slurm_task_manager.submit(
            name="test_cancel_task",
            entrypoint_path="nvidia-smi; sleep 1; echo notok")
print(task_id)
!/data/slurm/bin/squeue -u hongning.zhang

42691
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
             42691       gpu test_can hongning PD       0:00      1 (None)
